In [50]:
from io import StringIO # Extract
import requests # Extract
from bs4 import BeautifulSoup # Extract
import pandas as pd # Transformation
import sqlite3 # Load
from datetime import datetime # For Time of Log
from icecream import ic

In [3]:
a=200
b=250
c=300

for i in range(1,5): 
    ans = a*b/c
    a -= 10
    b +=20
    print(a)
    print(b)
    print(ans)

190
270
166.66666666666666
180
290
171.0
170
310
174.0
160
330
175.66666666666666


In [5]:
a=200
b=250
c=300

for i in range(1,5): 
    ans = ic(a*b/c)
    a -= 10
    b +=20

ic| Error: Failed to access the underlying source code for analysis. Was ic() invoked in a REPL (e.g. from the command line), a frozen application (e.g. packaged with PyInstaller), or did the underlying source code change during execution?
ic| Error: Failed to access the underlying source code for analysis. Was ic() invoked in a REPL (e.g. from the command line), a frozen application (e.g. packaged with PyInstaller), or did the underlying source code change during execution?
ic| Error: Failed to access the underlying source code for analysis. Was ic() invoked in a REPL (e.g. from the command line), a frozen application (e.g. packaged with PyInstaller), or did the underlying source code change during execution?
ic| Error: Failed to access the underlying source code for analysis. Was ic() invoked in a REPL (e.g. from the command line), a frozen application (e.g. packaged with PyInstaller), or did the underlying source code change during execution?


In [14]:
# Step 0
def log_progress(message):
    with open('./logs/code_log.txt', 'a') as f:
        f.write(f'{datetime.now()}: {message}.\n')

In [19]:
# Step 1
def extract(url, table_att):
    soup = requests.get(url).text
    web = BeautifulSoup(soup, 'html.parser')
    table = web.find('span', string=table_att).find_next('table')
    df = pd.read_html(StringIO(str(table)))[0]
    
    log_progress('Data Extracted Successfully from Wikipedia')
    
    return df

In [16]:
# Step 2
def transformation(df, csv_path):
    exchange_rate = pd.read_csv(csv_path, index_col = 0).to_dict()['Rate']
    
    
    df['MC_GBP_Billion'] = round(df['Market cap (US$ billion)']*exchange_rate['GBP'],2)
    df['MC_EUR_Billion'] = round(df['Market cap (US$ billion)']*exchange_rate['EUR'],2)
    df['MC_INR_Billion'] = round(df['Market cap (US$ billion)']*exchange_rate['INR'],2)
    
    print(df['MC_EUR_Billion'][4])
    
    log_progress('Data Transformation Done Now, Step 2 Completed.')
    
    return df

In [9]:
# Step 3 Load
# To CSV File
def load_to_csv(df, out_path):
    df.to_csv(out_path)
    
    log_progress('Data Loaded into the CSV File')

In [34]:
# Step 3
# SQLite
def load_to_db(df, sql_conn, tn):
    df.to_sql(tn, sql_conn, if_exists='replace', index=False)
    
    log_progress('Data Loaded into the SQLite Database')

In [35]:
if __name__ == '__main__':
    url = 'https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks'
    output_csv_path = './output/Largest_banks_data.csv'
    database_name = './output/Banks.db'
    table_name = 'Largest_banks'
    
    log_progress('Preliminaries complete. Initiating ETL process')
    
    df = extract(url, 'By market capitalization')
    log_progress('Extraction Completed')
    
    df = transformation(df, "./input/exchange_rate.csv")
    log_progress('Data Transformed Successfully')
    print(df)
    
    load_to_csv(df, output_csv_path)
    log_progress('DataFrame has been uploaded into the CSV file on our PC')
    
    with sqlite3.connect(database_name) as conn:
        load_to_db(df, conn, table_name)
    log_progress('DataFrame has been uploaded into the SQLite Database Successfully')

146.86
   Rank                                Bank name  Market cap (US$ billion)  \
0     1                           JPMorgan Chase                    432.92   
1     2                          Bank of America                    231.52   
2     3  Industrial and Commercial Bank of China                    194.56   
3     4               Agricultural Bank of China                    160.68   
4     5                                HDFC Bank                    157.91   
5     6                              Wells Fargo                    155.87   
6     7                        HSBC Holdings PLC                    148.90   
7     8                           Morgan Stanley                    140.83   
8     9                  China Construction Bank                    139.82   
9    10                            Bank of China                    136.81   

   MC_GBP_Billion  MC_EUR_Billion  MC_INR_Billion  
0          346.34          402.62        35910.71  
1          185.22          215